# Consigna
 def UserForGenre( genero : str ): 
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
"Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]} 

In [1]:
import pandas as pd

Para resolver este problema necesitare User_items y los Games

In [3]:
items_path = '../ETL/Data_Extracted/DataFrame_items.parquet'
ItemsDF = pd.read_parquet(items_path)

In [5]:
games_path = '../ETL/Data_Extracted/DataFrame_games.parquet'
GamesDF = pd.read_parquet(games_path)

In [11]:
# de aqui me interesan item_id user_id playtime_forever
ItemsDF.head(1)

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


En base al DF anterior no puedo saber con exactitud cuantas horas jugo por año en el genero dado, lo que si puedo dar el total por genero

In [12]:
# de aqui me interesan item_id user_id playtime_forever
ItemsDF = ItemsDF[['item_id','playtime_forever','user_id']]
ItemsDF

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
1,20,0.0,76561197970982479
2,30,7.0,76561197970982479
3,40,0.0,76561197970982479
4,50,0.0,76561197970982479
...,...,...,...
5153204,346330,0.0,76561198329548331
5153205,373330,0.0,76561198329548331
5153206,388490,3.0,76561198329548331
5153207,521570,4.0,76561198329548331


In [13]:
# De aqui me interesan id del juego, los generos a los que pertenece, genre release_date
GamesDF = GamesDF[['genres','id']]
GamesDF

,genres,id
0,"[Action, Casual, Indie, Simulation, Strategy]",761140
1,"[Free to Play, Indie, RPG, Strategy]",643980
2,"[Casual, Free to Play, Indie, Simulation, Sports]",670290
3,"[Action, Adventure, Casual]",767400
4,"[Action, Adventure, Simulation]",772540
...,...,...
27457,"[Action, Adventure, Casual, Indie]",745400
27458,"[Casual, Indie, Simulation, Strategy]",773640
27459,"[Casual, Indie, Strategy]",733530
27460,"[Indie, Racing, Simulation]",610660


In [14]:
#Cambio el nombre para poder unirlos por esa columna en comun
GamesDF = GamesDF.rename(columns={'id': 'item_id'})

In [16]:
#Agrego el precio a la informacion de mis usuarios
UserDataDF = pd.merge(GamesDF, ItemsDF, on='item_id' , how='left')

In [18]:
# Agrego las columnas que perdi en el merge
UserDataDF['playtime_forever'] = ItemsDF['playtime_forever']

In [19]:
UserDataDF['user_id'] = ItemsDF['user_id']

In [20]:
UserDataDF

,genres,item_id,playtime_forever,user_id
0,"[Action, Casual, Indie, Simulation, Strategy]",761140,6.0,76561197970982479
1,"[Free to Play, Indie, RPG, Strategy]",643980,0.0,76561197970982479
2,"[Casual, Free to Play, Indie, Simulation, Sports]",670290,7.0,76561197970982479
3,"[Action, Adventure, Casual]",767400,0.0,76561197970982479
4,"[Action, Adventure, Simulation]",772540,0.0,76561197970982479
...,...,...,...,...
4116742,"[Action, Adventure, Casual, Indie]",745400,6192.0,76561198025915551
4116743,"[Casual, Indie, Simulation, Strategy]",773640,2953.0,76561198025915551
4116744,"[Casual, Indie, Strategy]",733530,2093.0,76561198025915551
4116745,"[Indie, Racing, Simulation]",610660,112.0,76561198025915551


In [21]:
#pivoteo en la columna de generos
generos = UserDataDF.explode(['genres'])
generos

,genres,item_id,playtime_forever,user_id
0,Action,761140,6.0,76561197970982479
0,Casual,761140,6.0,76561197970982479
0,Indie,761140,6.0,76561197970982479
0,Simulation,761140,6.0,76561197970982479
0,Strategy,761140,6.0,76561197970982479
...,...,...,...,...
4116745,Indie,610660,112.0,76561198025915551
4116745,Racing,610660,112.0,76561198025915551
4116745,Simulation,610660,112.0,76561198025915551
4116746,Casual,658870,0.0,76561198025915551


In [22]:
# Extraigo dummies para saber a que generos pertenece un Game
generos = pd.get_dummies(generos['genres'])
generos

,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116745,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4116745,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4116745,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4116746,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
generos = generos.replace({True: 1, False: 0})
generos

C:\Users\ahurt\AppData\Local\Temp\ipykernel_15768\2540680754.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  generos = generos.replace({True: 1, False: 0})


,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4116745,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4116745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4116746,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#Elimino del DF la columna Genre ya no la necesito mas
UserDataDF = UserDataDF.drop(columns=['genres'])


In [28]:
#Concateno con el dataframe que genere
UserDataDF = pd.concat([UserDataDF, generos], axis=1)

In [29]:
UserDataDF

,item_id,playtime_forever,user_id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,761140,6.0,76561197970982479,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,761140,6.0,76561197970982479,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
0,761140,6.0,76561197970982479,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,761140,6.0,76561197970982479,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
0,761140,6.0,76561197970982479,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116745,610660,112.0,76561198025915551,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4116745,610660,112.0,76561198025915551,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4116745,610660,112.0,76561198025915551,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4116746,658870,0.0,76561198025915551,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
def UserForGenre(genero: str):
    # Verificar si el género está presente en las columnas del DataFrame
    if genero in UserDataDF.columns:
        # Filtrar el DataFrame para el género dado
        genre_df = UserDataDF[UserDataDF[genero] == 1]

        # Agrupar por usuario y sumar las horas jugadas
        user_playtime = genre_df.groupby('user_id')['playtime_forever'].sum()

        # Encontrar el usuario con el máximo de horas jugadas
        max_user = user_playtime.idxmax()

       

        return max_user
    else:
        return (f"El género '{genero}' no está presente en el DataFrame.")

In [ ]:
# prueba 
print(UserForGenre('Action'))